# CS202: Compiler Construction

## In-class Exercises, Week of 02/01/2021

----
# PART I: Abstract Syntax Trees

## Question 1

The following grammar defines the *concrete syntax* for a language of integer arithmetic (numbers and the "plus" operator).

\begin{align*}
exp &::= n \\
&\mid exp + exp
\end{align*}

The following class hierarchy defines an *abstract syntax* for the same language.

In [1]:
from cs202_support.base_ast import AST, print_ast
from dataclasses import dataclass

@dataclass
class RIntExp(AST):
    pass

@dataclass
class Int(RIntExp):
    val: int

@dataclass
class Plus(RIntExp):
    e1: RIntExp
    e2: RIntExp

Write an abstract syntax tree for the expression `1 + 2 + 3`.

In [2]:
ast = Plus(Int(1), Plus(Int(2), Int(3)))
print(print_ast(ast))

Plus(
 Int(1),
 Plus(
  Int(2),
  Int(3)))


## Question 2

The code below defines a parser that transforms concrete syntax for this simple language into abstract syntax trees.

In [8]:
from lark import Lark
_rint_parser = Lark(r"""
    ?exp: NUMBER -> int_e
        | exp "+" exp -> plus_e
        | "(" exp ")"
    %import common.NUMBER
    %import common.CNAME
    %import common.WS
    %ignore WS
    """, start='exp')

def parse(s):
    def t_ast(e):
        if e.data == 'int_e':
            return Int(int(e.children[0]))
        elif e.data == 'plus_e':
            e1, e2 = e.children
            return Plus(t_ast(e1), t_ast(e2))

    parsed = _rint_parser.parse(s)
    ast = t_ast(parsed)
    return ast

Write code to use the parser above to parse the expression `1 + 2 + 3` into an abstract syntax tree.

In [4]:
ast = parse('1 + 2 + 3')
print(print_ast(ast))

Plus(
 Int(1),
 Plus(
  Int(2),
  Int(3)))


In [5]:
from ast import parse
parse('print(3)')

## Question 3

Write an *interpreter* for this language.

**The structure of your function should follow the structure of the AST**

In [6]:
def eval_rint(e: RIntExp) -> int:
    # structure should match AST structure
    # 1. want match case for each AST node type
    # 2. want to make recursive calls for every subexpression
    match e:
        case Int(i):
            # i => e.val
            return i
        case Plus(e1, e2):
            # e1 => e.e1, e2 => e.e2
            return eval_rint(e1) + eval_rint(e2)

In [9]:
# TEST CASE
assert eval_rint(parse('1 + 2 + 3')) == 6
assert eval_rint(parse('42 + 20 + 10 + 5 + 5')) == 82

----
# PART II: x86 Assembly

In [10]:
from cs202_support.eval_x86 import X86Emulator

## Question 4

Write x86 assembly code to add the numbers 1 and 2, putting the result in the register `rax`.

In [11]:
emu = X86Emulator(logging = False)
emu.eval_instructions("""
movq $1, %rax
addq $2, %rax""")

,Location,Old,New
0,reg rax,None,3


## Question 5 

Write x86 assembly code to add the numbers 1, 2, 3, and 4, putting the result in the register `rdi`.

In [12]:
emu = X86Emulator(logging = False)
emu.eval_instructions("""
movq $1, %rdi
addq $2, %rdi
addq $3, %rdi
addq $4, %rdi""")

,Location,Old,New
0,reg rdi,None,10


## Question 6

Write a complete x86 program to:

- Place the number 42 in the register `rdi`
- Call the function `print_int` in the runtime
- Return cleanly to the operating system

Hint: try using the [Assignment 1 online compiler](https://jnear.w3.uvm.edu/cs202/compiler-a1.php).

In [13]:
emu = X86Emulator(logging = False)
emu.eval_program("""
.globl main
main:
  pushq %rbp
  movq %rsp, %rbp
  jmp start
start:
  movq $42, %rdi
  callq print_int
  jmp conclusion
conclusion:
  movq $0, %rax
  popq %rbp
  retq""")

[42]

## Question 7

Write code to generate a *pseudo-x86 abstract syntax tree* for the `start` block in the program above.

Hint: reference the [pseudo-x86 AST class hierarchy](https://github.com/jnear/cs202-assignments/blob/master/cs202_support/x86exp.py). Debug your solution using the online compiler's output for the `select instructions` pass.

In [34]:
import cs202_support.x86exp as x86

ast = x86.Program({
    'start': []
})

print(print_ast(ast))

Program({
 'start':
  [
   Movq(
    Int(42),
    Reg(rax)),
   Jmp(conclusion)
  ]
})
